In [5]:
from siamese_utils import PseudoSiamese
from PIL import Image
import pandas as pd
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from torch.autograd import Variable

In [6]:
class mydataset(object):

	def __init__(self, x0, x1, label):
		self.size = len(label)
		self.x0 = torch.from_numpy(x0)
		self.x1 = torch.from_numpy(x1)
		self.label = label

	def __getitem__(self, index):
		return (self.x0[index],
				self.x1[index],
				self.label[index])

	def __len__(self):
		return self.size

In [7]:
filenames_1 = ['f03.tif','f04.tif','f05.tif','f06.tif','f07.tif','f08.tif','f09.tif','f10.tif','f11.tif','f12.tif',
'f13.tif','f14.tif','f15.tif','f16.tif','f17.tif','f18.tif','f19.tif','f20.tif','f21.tif','f22.tif']
filenames_2 = ['f03.tif','f04.tif','f05.tif','f06.tif','f07.tif','f08.tif','f09.tif','f10.tif','f11.tif','f12.tif',
'f13.tif','f14.tif','f15.tif','f16.tif','f17.tif','f18.tif','f19.tif','f20.tif','f21.tif','f22.tif']
labels = ['ab','ab','ab','ab','ab','veh','veh','veh','veh','veh',
'ab','ab','ab','ab','ab','veh','veh','veh','veh','veh']

train_filenames = ['f03.tif','f04.tif','f05.tif','f06.tif','f07.tif','f08.tif','f09.tif','f10.tif','f11.tif','f12.tif']
train_labels = ['ab','ab','ab','ab','ab','veh','veh','veh','veh','veh']

valid_filenames_1 = ['f13.tif','f14.tif','f15.tif','f16.tif','f17.tif','f18.tif','f19.tif','f20.tif','f21.tif','f22.tif']
valid_filenames_2 = ['f13.tif','f14.tif','f15.tif','f16.tif','f17.tif','f18.tif','f19.tif','f20.tif','f21.tif','f22.tif']
valid_labels = ['ab','ab','ab','ab','ab','veh','veh','veh','veh','veh']

df = pd.DataFrame({'filenames':filenames_1, 'labels':labels})

In [8]:
# creating pairs
x0_data = []
x1_data = []
for i in range(len(filenames_1)):
    img_1 = cv2.imread('/Users/iron4dam/Desktop/Neuron-Image-Classification/training_data/'+filenames_1[i])
    img_2 = cv2.imread('/Users/iron4dam/Desktop/Neuron-Image-Classification/training_data/'+filenames_2[i])
    x0_data.append(img_1/255.)
    x1_data.append(img_2/255.)

In [9]:
x0_data = np.array(x0_data, dtype=np.float32)
x1_data = np.array(x1_data, dtype=np.float32)

In [10]:
x0_data.shape

(20, 2048, 2048, 3)

In [11]:
x0_data = x0_data.reshape([-1,3,2048,2048])
x1_data = x1_data.reshape([-1,3,2048,2048])

In [12]:
labels_new = pd.factorize(labels)[0]
labels_new = labels_new/1.

In [13]:
train_dataset = mydataset(x0_data,x1_data,labels_new)
test_dataset = train_dataset

In [14]:
model = PseudoSiamese()

In [15]:
train_loader = torch.utils.data.DataLoader(train_dataset)
test_loader = torch.utils.data.DataLoader(test_dataset)

In [16]:
learning_rate = 0.01
momentum = 0.9
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,
                                momentum=momentum)

In [17]:
train_loss = []
model.train()
for batch_idx, (x0, x1, labels) in enumerate(train_loader):
    x0, x1, labels = Variable(x0), Variable(x1), Variable(labels)
    output1, output2 = model(x0, x1)

RuntimeError: size mismatch, m1: [1 x 131072], m2: [512 x 512] at /Users/distiller/project/conda/conda-bld/pytorch_1556653492823/work/aten/src/TH/generic/THTensorMath.cpp:961